In [3]:
import torch
import torch.nn as nn

 # This thing should be tried
        # for i in range(x.size(1)):
        #     if tensor is None:
        #         tensor = self.layer(x[:, i:i+1, :, :])
        #     else:
        #         tensor = torch.cat((tensor, self.layer(x[:, i:i+1, :, :])), 1)

# Ideas
- conv + fc >>> add all channels
- conv + add all channels >>> fc

In [ ]:
class Model_1(nn.Module):
    def __init__(self):
        super(Model_1, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=5, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU()
        )

    def forward(self, x):
        tensor = None
        for i in range(x.size(1)):
            if tensor is None:
                tensor = self.layer(x[:, i:i+1, :, :])
            else:
                tensor = tensor + self.layer(x[:, i:i+1, :, :])

        
           
        return tensor